Construir un analizador de sentimientos que utilice la API de OPENAI o textblob respetando los principios SOLID y utilizando POO.

In [6]:
from abc import ABC, abstractmethod

class AnalizadorDeSentimientos(ABC):
    
    @abstractmethod
    def analizar_sentimiento(self, texto):
        pass

In [28]:
class AnalizadorDeSentimientosOpenAI(AnalizadorDeSentimientos):
    def analizar_sentimiento(self, texto):
        import openai
        from dotenv import load_dotenv
        import os

        load_dotenv()
        openai.api_key = os.environ.get("OPENAI_API_KEY")

        system = """Asume que eres un bot que analiza sentimientos presentes en un texto ficticio.
            Te entrego el texto y necesito que me entregues
            como respuesta exclusivamente un float de máximo 4 caracteres entre
            el valor -1 si es negativo máximo, 1 positivo máximo, 0 neutro."""

        messages = [{"role": "system", "content": system}, 
                    {"role": "user", "content": texto}]

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
        )
        return float(response.choices[0].message["content"])

In [24]:

class AnalizadorDeSentimientosTextBlob(AnalizadorDeSentimientos):

    def analizar_sentimiento(self, texto):
        from textblob import TextBlob
        analisis = TextBlob(texto)
        return analisis.sentiment.polarity 

In [12]:
class Sentimiento:
    def __init__(self, nombre, color):
        self.nombre = nombre
        self.color = color
    
    def __str__(self):
        from termcolor import colored
        return colored(self.nombre, self.color)

In [13]:
class ClasificadorDeSentimiento:
    def __init__(self, rangos):
        self.rangos = rangos

    def clasificar_sentimiento(self, polaridad):
        # si la polaridad se halla entre los rangos, toma el sentimiento
        for rango, sentimiento in self.rangos:
            if rango[0] < polaridad <= rango[1]:
                return sentimiento
        return Sentimiento("Muy negativo", "red")

In [20]:
rangos = [
    ((-1, -0.75), Sentimiento("Muy negativo", "red")),
    ((-0.75, -0.5), Sentimiento("Negativo", "yellow")),
    ((-0.5, -0.25), Sentimiento("Levemente negativo", "blue")),
    ((-0.25, 0.25), Sentimiento("Neutral", "cyan")),
    ((0.25, 0.5), Sentimiento("Levemente positivo", "green")),
    ((0.5, 0.75), Sentimiento("Positivo", "magenta")),
    ((0.75, 1), Sentimiento("Muy positivo", "white")),
    ]

In [31]:
texto = str(input("Ingrese un texto: "))

analizador = AnalizadorDeSentimientosTextBlob()
polaridad = analizador.analizar_sentimiento(texto)
print("La polaridad es: ", polaridad)

clasificador = ClasificadorDeSentimiento(rangos)
clasificacion = clasificador.clasificar_sentimiento(polaridad)
print("La clasificación es: ", clasificacion)

La polaridad es:  0.8
La clasificación es:  Muy positivo
